# Post Processing
1. Backup grading result.
2. Generate score report.
3. Create individual scored pdf.
4. Collect samples.
5. Email the individual scored pdf.

In [2]:
pdf_file = "../data/TestScript.pdf"

In [28]:
import os

file_name = os.path.basename(pdf_file)
file_name = os.path.splitext(file_name)[0]
base_path = "../marking_form/" + file_name
base_path_images = base_path + "/images/"
base_path_annotations = base_path+"/annotations/"
base_path_questions = base_path+"/questions"

base_path_marked_images = base_path + "/marked/images/"
os.makedirs(base_path_marked_images, exist_ok=True)

### Remove version history
Before you backup.

In [5]:
import os
for path, currentDirectory, files in os.walk(base_path_questions):
    for file in files:
        if file.startswith("control-") or file.startswith("mark-"):
            os.remove(os.path.join(path, file))

Zip the website.

In [6]:
import shutil
shutil.make_archive(base_path,"zip",base_path)

'/workspaces/ai-scoring-handwriting-assignment/marking_form/TestScript.zip'

# Generate Score Report

You must visit the ID and Name pages to verify the values before generate the marksheet.

In [27]:
import os
import json
import pandas as pd
from termcolor import colored

pageToStudentId = {}

# read base_path_annotations
with open(os.path.join(base_path_annotations, "annotations.json")) as f:
    data = json.load(f)
    # data is a dict and get the number of items
numberOfPage = len(data)

with open(os.path.join(base_path_questions, "ID", "mark.json")) as f:
    data = json.load(f)
    for i in data:
        pageToStudentId[i["id"]] = (
            i["overridedMark"] if i["overridedMark"] != "" else i["mark"]
        )

def getStudentId(page):
    # search reverse for the student ID page.
    for p in range(page, page - numberOfPage, -1):
        if str(p) in pageToStudentId:
            return pageToStudentId[str(p)]
    print(colored("{} is not in pageToStudentId!".format(page), "red"))
    return None

questionAndMarks = {}
for path, currentDirectory, files in os.walk(base_path_questions):
    for file in files:
        if file == "mark.json":
            question = path[len(base_path_questions) + 1 :]
            f = open(os.path.join(path, file))
            data = json.load(f)
            marks = {}
            for i in data:
                studentId = getStudentId(int(i["id"]))
                marks[studentId] = (
                    i["overridedMark"] if i["overridedMark"] != "" else i["mark"]
                )
            questionAndMarks[question] = marks
            f.close()
marksDf = pd.DataFrame(questionAndMarks)
marksDf = marksDf[
    ["ID", "NAME", "CLASS"]
    + [
        col
        for col in sorted(marksDf.columns)
        if col != "ID" and col != "NAME" and col != "CLASS"
    ]
]

marksDf["Marks"] = (
    marksDf.loc[:, ~marksDf.columns.isin(["ID", "NAME", "CLASS"])]
    .apply(pd.to_numeric)
    .sum(axis=1)
)
print(marksDf)

                  ID           NAME CLASS Q1 Q36 Q37 Q38  Marks
200275822  200275822    LEUNGTszHin        0   0   0   0      0
200241763  200241763     TAMHinChun        0   0   0   0      0
200238088  200238088      LoChunHei     A  1   0   0   0      1
200186026  200186026   YUEN WingChu   E&L  0   0   0   0      0
200091406  200091406    CHEUNGWaKin   E&L  0   0   0   0      0
200035120  200035120   TSANGYukKuer     A  0   1   0   0      1
200177460  200177460    YIMWaiChung   EaL  0   0   0   0      0
200081960  200081960    HUNGKamFung   E&L  0   0   0   0      0
200365446  200365446    CHEUNGHoShi     A  0   0   0   0      0
200429237  200429237      LUOMianan     A  0   0   0   0      0
200144594  200144594    CHANPakHong        0   0   0   0      0
200276566  200276566     NGChunWing        0   0   0   0      0
200167780  200167780        LOPakHo   ESL  0   0   0   0      0
200133094  200133094       SOKaChun     A  0   0   0   0      0
200532530  200532530  KWAN Wing Sze     

# Create Scored Scripts

In [31]:
import shutil
import os

if os.path.exists(base_path_marked_images):
    shutil.rmtree(base_path_marked_images)
shutil.copytree(base_path_images, base_path_marked_images)

'../marking_form/TestScript/marked/images/'

In [35]:
import json
annotations_path = base_path_annotations + "annotations.json"
with open(annotations_path, "r") as f: 
    annotations = json.load(f)          

#flatten annotations to list 
annotations_list = []
for page in annotations:
    for annotation in annotations[page]:
        annotation["page"] = int(page)
        # x to left, y to top
        annotation["left"] = annotation["x"]
        annotation["top"] = annotation["y"]
        annotation.pop("x")
        annotation.pop("y")
        annotations_list.append(annotation) 
annotations_list

# convert annotations_list to dict with key with label
annotations_dict = {}
for annotation in annotations_list:
    annotations_dict[annotation["label"]] = annotation
annotations_dict


{'NAME': {'width': 322,
  'height': 77,
  'label': 'NAME',
  'page': 0,
  'left': 238,
  'top': 173},
 'ID': {'width': 471,
  'height': 83,
  'label': 'ID',
  'page': 0,
  'left': 175,
  'top': 259},
 'CLASS': {'width': 168,
  'height': 91,
  'label': 'CLASS',
  'page': 0,
  'left': 835,
  'top': 248},
 'Q1': {'width': 322,
  'height': 111,
  'label': 'Q1',
  'page': 0,
  'left': 164,
  'top': 344},
 'Q36': {'width': 347,
  'height': 138,
  'label': 'Q36',
  'page': 1,
  'left': 184,
  'top': 482},
 'Q37': {'width': 410,
  'height': 150,
  'label': 'Q37',
  'page': 1,
  'left': 665,
  'top': 460},
 'Q38': {'width': 384,
  'height': 143,
  'label': 'Q38',
  'page': 1,
  'left': 1166,
  'top': 473}}

In [34]:
studentIdToPage={}
with open(os.path.join(base_path_questions, "ID", "mark.json")) as f:
    data = json.load(f)
    for i in data:
        studentId = i["overridedMark"] if i["overridedMark"] != "" else i["mark"]
        studentIdToPage[studentId] = int(i["id"])
studentIdToPage


{'200012886': 0,
 '200238088': 10,
 '200021312': 100,
 '200017034': 102,
 '200237504': 104,
 '200232139': 106,
 '200005883': 108,
 '200532530': 110,
 '200144594': 112,
 '200048545': 12,
 '200072256': 14,
 '200266235': 16,
 '200119573': 18,
 '200365446': 2,
 '200096278': 20,
 '200084507': 22,
 '200169933': 24,
 '200236532': 26,
 '200149381': 28,
 '200143500': 30,
 '200188622': 32,
 '200185933': 34,
 '200167780': 36,
 '200189249': 38,
 '200147191': 4,
 '200337380': 40,
 '200186026': 42,
 '200429237': 44,
 '200159097': 46,
 '200529119': 48,
 '200099220': 50,
 '200169146': 52,
 '200169171': 54,
 '200133094': 56,
 '200276566': 58,
 '200035132': 6,
 '200091406': 60,
 '200006867': 62,
 '200035120': 64,
 '200275822': 66,
 '200172741': 68,
 '200082391': 70,
 '200086758': 72,
 '200241763': 74,
 '200177527': 76,
 '200023631': 78,
 '200177460': 8,
 '200270390': 80,
 '200228716': 82,
 '200084821': 84,
 '200187164': 86,
 '200081960': 88,
 '200148151': 90,
 '200277378': 92,
 '200358399': 94,
 '200239